# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,21.14,65,72,0.45,US,1724279205
1,1,cabedelo,-6.9811,-34.8339,25.37,86,8,6.14,BR,1724279206
2,2,tuam,53.5167,-8.8500,14.82,84,100,9.53,IE,1724279207
3,3,dongcun,36.7767,121.1597,24.24,92,100,2.87,CN,1724279208
4,4,oum el bouaghi,35.8754,7.1135,20.43,49,82,1.41,DZ,1724279209


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True, 
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    scale= 0.5,
    size="Humidity"
)

# Display the map
map_plot_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
good_city_data_df = city_data_df.loc[(city_data_df['Max Temp']< 27) & (city_data_df['Max Temp']>21)
&(city_data_df['Wind Speed']< 4.5) & (city_data_df['Cloudiness']== 0)]

# Drop any rows with null values
great_city_data_df= good_city_data_df.dropna()

# Display sample data
great_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
112,112,hecun,36.5333,114.1111,23.60,71,0,2.26,CN,1724279343
145,145,turan,16.0678,108.2208,24.99,94,0,0.51,VN,1724279385
151,151,kone,-21.0595,164.8658,23.22,51,0,3.53,NC,1724279393
162,162,minna,9.6139,6.5569,24.84,88,0,1.54,NG,1724279406
182,182,gazojak,41.1875,61.4036,23.85,25,0,3.86,TM,1724279433
185,185,chefchaouene,35.1688,-5.2636,23.44,72,0,0.23,MA,1724279437
188,188,barberino di mugello,44.0027,11.2371,23.68,70,0,1.34,IT,1724279440
248,248,bastia,42.6667,9.3333,23.50,85,0,1.54,FR,1724279520
266,266,nova vicosa,-17.8919,-39.3719,22.90,84,0,3.06,BR,1724279542
301,301,xacmaz,41.4635,48.8060,23.48,43,0,1.24,AZ,1724279586


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = great_city_data_df.copy(deep=True)
hotel_df = hotel_df[['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
112,hecun,CN,36.5333,114.1111,71,
145,turan,VN,16.0678,108.2208,94,
151,kone,NC,-21.0595,164.8658,51,
162,minna,NG,9.6139,6.5569,88,
182,gazojak,TM,41.1875,61.4036,25,
185,chefchaouene,MA,35.1688,-5.2636,72,
188,barberino di mugello,IT,44.0027,11.2371,70,
248,bastia,FR,42.6667,9.3333,85,
266,nova vicosa,BR,-17.8919,-39.3719,84,
301,xacmaz,AZ,41.4635,48.8060,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"


params = {
    "categories":categories,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
   
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hecun - nearest hotel: 响堂云居酒店
turan - nearest hotel: Thanh Thanh Hotel
kone - nearest hotel: Pacifik Appartels
minna - nearest hotel: Haske Main Hotel
gazojak - nearest hotel: No hotel found
chefchaouene - nearest hotel: Hotel Andalouz
barberino di mugello - nearest hotel: Sant’Andrea Country Cottage
bastia - nearest hotel: Adonis Saint Florent
nova vicosa - nearest hotel: Pousada Pontal da Barra
xacmaz - nearest hotel: Xaçmaz Otel
itapuranga - nearest hotel: No hotel found
ponte nova - nearest hotel: No hotel found
xinyang - nearest hotel: 德龙太子酒店
hammond - nearest hotel: Comfort Inn Hammond I-94
markaz-e hukumat-e darweshan - nearest hotel: No hotel found
curvelo - nearest hotel: Hotel Bandeirante
greenville - nearest hotel: Quality Inn
tivat - nearest hotel: Regent
lagoa formosa - nearest hotel: Hotel HS
pikeville - nearest hotel: Hampton Inn
fort bragg - nearest hotel: Airborne Inn Lodging
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
conde - nearest hotel: Pa

,City,Country,Lat,Lng,Humidity,Hotel Name
112,hecun,CN,36.5333,114.1111,71,响堂云居酒店
145,turan,VN,16.0678,108.2208,94,Thanh Thanh Hotel
151,kone,NC,-21.0595,164.8658,51,Pacifik Appartels
162,minna,NG,9.6139,6.5569,88,Haske Main Hotel
182,gazojak,TM,41.1875,61.4036,25,No hotel found
185,chefchaouene,MA,35.1688,-5.2636,72,Hotel Andalouz
188,barberino di mugello,IT,44.0027,11.2371,70,Sant’Andrea Country Cottage
248,bastia,FR,42.6667,9.3333,85,Adonis Saint Florent
266,nova vicosa,BR,-17.8919,-39.3719,84,Pousada Pontal da Barra
301,xacmaz,AZ,41.4635,48.8060,43,Xaçmaz Otel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_city_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True, 
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    scale= 0.5,
    hover_cols=['Lng', 'Lat','City', 'Humidity','Hotel Name', 'Country']
    
)

# Display the map
map_plot_city_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)